## 1. Daten sammeln


Smartvote stellt eine Schnittstelle (API) zur Verfügung, über welche Daten abgefragt werden können. Wir wollen die Antworten aller Kandidierenden der letzten Nationalratswahlen abfragen und die Resultate speichern.

Um mit Python eine Anfrage an eine Schnittstelle im Internet zu senden, wird die Bibliothek requests verwendet. Sie ermöglicht das Zusammenstellen und Versenden der Anfrage sowie das Verarbeiten der erhaltenen Antwort.

Die Daten werden in [JSON](https://de.wikipedia.org/wiki/JavaScript_Object_Notation) zur Verfügung gestellt, einem simplen, strukturierten Datenformat.


Zuerst müssen die verwendeten Libraries importiert werden.

- `requests` für das Anfordern der Daten
- `json` für das Handhaben der gesammelten Daten


In [ ]:
import requests
import json

Zum Zwecke der heutigen Übung verwenden wir die Daten der letzten Nationalratswahlen.

In [ ]:
mode = 1
# 1 = Nationalratswahlen

electionId = "1057" 
filename = "nationalrat_raw.json"

Danach kann die Schnittstelle angefragt werden. Über die Schnittstelle holen wir zu jeder Person die maximale Information.


In [ ]:
url = "https://backend.smartvote.ch/api/graphql"
headers = {}

body = {
    "operationName": "SearchCandidates",
    "variables": {"searchParams": {"electionId": electionId}},
    "query": """
    query SearchCandidates($searchParams: CandidateSearchParams!) {
        candidates(searchParams: $searchParams) {
            ...SearchCandidate
        }
    }

    fragment SearchCandidate on Candidate {
        id
        firstname
        lastname
        yearOfBirth
        profileImageUrl
        isIncumbent
        isElected
        partyAbbreviation
        partyColor
        hasSmartvoteProfile
        gender

        answers {
            ...Answer
        }
    }

    fragment Answer on Answer {
        questionId
        value
    }
    """,
}


response = requests.post(url, headers=headers, json=body)


if response.status_code == 200:
    candidates = response.json()["data"]["candidates"]
    print(f"Request was successful and returned {len(candidates)} candidate(s)")
else:
    print(f"Request failed with status code {response.status_code}")

Jetzt können wir uns die Informationen zu einer beliebigen Kandidierenden Person darstellen.


In [ ]:
print(json.dumps(candidates[242 if len(candidates) > 242 else 52], indent=4, ensure_ascii=False))

Als Nächstes entfernen wir alle Kandidat\*innen aus, die kein Smartvote-Profil haben.


In [ ]:
candidates = [candidate for candidate in candidates if candidate["hasSmartvoteProfile"]]

Jetzt kombinieren wir diese Daten noch mit den tatsächlichen Fragen, welche uns von Smartvote zur Verfügung gestellt wurden:


In [ ]:
fragen = [   
    {'ID_question': '32214', 'category': '11451', 'language': '0', 'question': 'Befürworten Sie eine Erhöhung des Rentenalters (z.B. auf 67 Jahre)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32215', 'category': '11451', 'language': '0', 'question': 'Soll der Staat mehr Mittel für die Krankenkassen-Prämienverbilligung zur Verfügung stellen?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32216', 'category': '11451', 'language': '0', 'question': 'Bei Ehepaaren ist die Höhe der Rente heute auf 150% der maximalen individuellen AHV-Rente begrenzt (Plafonierung). Soll diese Begrenzung abgeschafft werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32217', 'category': '11451', 'language': '0', 'question': 'Im Rahmen der BVG-Reform sollen die Renten gekürzt werden (Senkung Mindestumwandlungssatz von 6.8% auf 6%). Befürworten Sie diese Massnahme?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32218', 'category': '11451', 'language': '0', 'question': 'Soll die bezahlte Elternzeit von heute 14 Wochen Mutterschafts- und zwei Wochen Vaterschaftsurlaub ausgebaut werden? [BePart-Frage]', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32219', 'category': '11451', 'language': '0', 'question': 'Soll der Bund den gemeinnützigen Wohnungsbau finanziell stärker fördern? [BePart-Frage]', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32220', 'category': '11452', 'language': '0', 'question': 'Soll für Kinder eine Impfpflicht gemäss dem Schweizerischen Impfplan eingeführt werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32221', 'category': '11452', 'language': '0', 'question': 'Befürworten Sie die Einführung einer Abgabe auf zuckerhaltige Lebensmittel (Zuckersteuer)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32222', 'category': '11452', 'language': '0', 'question': 'Sollen sich die Versicherten stärker an den Gesundheitskosten beteiligen (z.B. Erhöhung der Mindestfranchise)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32223', 'category': '11452', 'language': '0', 'question': 'Sollen in Zukunft bei Pandemien die Möglichkeiten des Bundesrats zur Einschränkung des Privat- und Wirtschaftslebens stärker begrenzt werden?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32224', 'category': '11452', 'language': '0', 'question': 'Soll der Bund die Kompetenz zur Festlegung des Spitalangebots erhalten (nationale Spitalplanung bzgl. Standorte und Leistungsangebot)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32225', 'category': '11453', 'language': '0', 'question': 'Gemäss dem Konzept der integrativen Schule werden Kinder mit Lernschwierigkeiten oder Behinderungen in regulären Schulklassen unterrichtet. Befürworten Sie dies?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32226', 'category': '11453', 'language': '0', 'question': 'Soll der Bund die Anforderungen an die gymnasiale Maturität erhöhen?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32227', 'category': '11453', 'language': '0', 'question': 'Soll sich der Staat stärker für gleiche Bildungschancen einsetzen (z.B. mit Förderunterricht-Gutscheinen für Schüler/-innen aus Familien mit geringem Einkommen)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32228', 'category': '11454', 'language': '0', 'question': 'Sollen die Anforderungen für Einbürgerungen gelockert werden (z.B. kürzere Wohnsitzfrist)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32229', 'category': '11454', 'language': '0', 'question': 'Sollen mehr qualifizierte Arbeitskräfte aus Nicht-EU/EFTA-Staaten in der Schweiz arbeiten dürfen (Erhöhung Drittstaatenkontingent)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32230', 'category': '11454', 'language': '0', 'question': 'Befürworten Sie Bestrebungen, Asylsuchende während des Asylverfahrens in Zentren ausserhalb Europas unterzubringen?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32231', 'category': '11454', 'language': '0', 'question': 'Sollen Ausländer/-innen, die seit mindestens zehn Jahren in der Schweiz leben, das Stimm- und Wahlrecht auf Gemeindeebene erhalten?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32232', 'category': '11455', 'language': '0', 'question': 'Soll der Konsum von Cannabis legalisiert werden? [BePart-Frage]', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32233', 'category': '11455', 'language': '0', 'question': 'Würden Sie es befürworten, wenn in der Schweiz die direkte aktive Sterbehilfe durch Ärztinnen und Ärzte straffrei möglich wäre?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32234', 'category': '11455', 'language': '0', 'question': 'Soll neben weiblich und männlich ein drittes amtliches Geschlecht eingeführt werden?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32235', 'category': '11455', 'language': '0', 'question': 'Finden Sie es richtig, dass gleichgeschlechtliche Paare in allen Bereichen heterosexuellen Paaren gleichgestellt sind?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32236', 'category': '11456', 'language': '0', 'question': 'Befürworten Sie Steuersenkungen auf Bundesebene in den nächsten vier Jahren?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32237', 'category': '11456', 'language': '0', 'question': 'Sollen Ehepaare getrennt als Einzelpersonen besteuert werden (Individualbesteuerung)? [BePart-Frage]', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32238', 'category': '11456', 'language': '0', 'question': 'Würden Sie die Einführung einer nationalen Erbschaftssteuer auf alle Erbschaften von über einer Million Franken befürworten? [BePart-Frage]', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32239', 'category': '11456', 'language': '0', 'question': 'Sollen die Unterschiede zwischen finanzstarken und -schwachen Kantonen durch den Finanzausgleich stärker verringert werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32240', 'category': '11457', 'language': '0', 'question': "Befürworten Sie die Einführung eines für alle Arbeitnehmenden gültigen Mindestlohnes von mindestens CHF 4'000 für eine Vollzeitstelle?", 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32241', 'category': '11457', 'language': '0', 'question': 'Befürworten Sie strengere Regulierungen für den Finanzsektor (z.B. strengere Eigenkapitalvorschriften für Banken, Bonusverbot)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32242', 'category': '11457', 'language': '0', 'question': 'Sollen Privathaushalte ihren Stromanbieter frei wählen können (vollständige Liberalisierung des Strommarktes)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32243', 'category': '11457', 'language': '0', 'question': 'Sollen die Vorschriften für den Bau von Wohnraum gelockert werden (z.B. Lärmschutz, Ausnutzungsziffern)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32244', 'category': '11457', 'language': '0', 'question': 'Befürworten Sie eine strengere Kontrolle der Lohngleichheit von Frauen und Männern?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32245', 'category': '11458', 'language': '0', 'question': 'Sollen stark befahrene Autobahnabschnitte ausgebaut werden?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32246', 'category': '11458', 'language': '0', 'question': 'Soll die Schweiz die Neuzulassung von Personenwagen mit Verbrennungsmotoren ab 2035 verbieten?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32247', 'category': '11458', 'language': '0', 'question': 'Soll zur Erreichung der Klimaziele ausschliesslich auf Anreize und Zielvereinbarungen anstatt auf Verbote und Einschränkungen gesetzt werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32248', 'category': '11458', 'language': '0', 'question': 'Finden Sie es richtig, dass für den Ausbau von erneuerbaren Energien die Vorschriften des Umwelt- und Landschaftsschutzes gelockert werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32249', 'category': '11458', 'language': '0', 'question': 'Sollen in der Schweiz wieder neue Atomkraftwerke gebaut werden dürfen?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32250', 'category': '11458', 'language': '0', 'question': 'Soll der Staat auch in ländlichen Regionen ein umfassendes Service-Public-Angebot garantieren?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32251', 'category': '11458', 'language': '0', 'question': 'Würden Sie die Einführung steigender Strompreise bei grösserem Verbrauch befürworten (progressive Stromtarife)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32252', 'category': '11459', 'language': '0', 'question': 'Befürworten Sie eine weitere Lockerung der Schutzbestimmungen für Grossraubtiere (Luchs, Wolf, Bär)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32253', 'category': '11459', 'language': '0', 'question': 'Sollen nur noch Landwirtschaftsbetriebe mit erweitertem ökologischen Leistungsnachweis Direktzahlungen erhalten?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32254', 'category': '11459', 'language': '0', 'question': 'Befürworten Sie strengere Tierschutzregelungen für die Haltung von Nutztieren (z.B. permanenter Zugang zum Aussenbereich)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32255', 'category': '11459', 'language': '0', 'question': 'Soll in der Schweiz 30 Prozent der Landesfläche zur Sicherung der Biodiversität zur Verfügung stehen? [BePart-Frage]', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32256', 'category': '11459', 'language': '0', 'question': 'Würden Sie ein Verbot von Einwegplastik und nicht-rezyklierbaren Kunststoffen befürworten? [BePart-Frage]', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32257', 'category': '11459', 'language': '0', 'question': 'Befürworten Sie staatliche Massnahmen für eine nachhaltigere Nutzung von elektronischen Geräten (z.B. Recht auf Reparatur, Verlängerung der Gewährleistungsfrist, garantierte Mindestzeit für Software-Updates)? [BePart-Frage]', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32258', 'category': '11460', 'language': '0', 'question': 'Soll das Schweizer Mobilfunknetz möglichst flächendeckend mit der neusten Technologie ausgestattet werden (aktuell 5G-Standard)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32259', 'category': '11460', 'language': '0', 'question': 'Soll der Bund im Bereich der Digitalisierung  staatlicher Dienstleistungen zusätzliche Kompetenzen erhalten, um gegenüber den Kantonen verbindliche Vorgaben und Standards durchsetzen zu können?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32260', 'category': '11460', 'language': '0', 'question': 'Befürworten Sie eine stärkere Regulierung grosser Internetplattformen (Transparenz-Regeln bei Algorithmen, verstärkte Haftung für Inhalte, Bekämpfung von Desinformation)?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32261', 'category': '11460', 'language': '0', 'question': 'Eine Volksinitiative möchte die Fernseh- und Radiogebühren reduzieren (200 Franken pro Haushalt, Abgabenbefreiung von Unternehmen). Unterstützen Sie dieses Anliegen?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32262', 'category': '11460', 'language': '0', 'question': 'Soll das Stimmrechtsalter auf 16 Jahre gesenkt werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32263', 'category': '11460', 'language': '0', 'question': 'Soll gegen Ausgaben des Bundes ab einer bestimmten Höhe ein Referendum ergriffen werden können (fakultatives Finanzreferendum)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32264', 'category': '11461', 'language': '0', 'question': "Befürworten Sie einen Ausbau des Armee-Sollbestandes auf mindestens 120'000 Soldat/-innen?", 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32265', 'category': '11461', 'language': '0', 'question': 'Soll die Schweizer Armee die Zusammenarbeit mit der NATO ausbauen?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32266', 'category': '11461', 'language': '0', 'question': 'Soll der Bundesrat in Fällen eines völkerrechtswidrigen Angriffskrieges (z.B. der Angriff auf die Ukraine) anderen Staaten die Wiederausfuhr von Schweizer Waffen erlauben dürfen? [BePart-Frage]', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32267', 'category': '11461', 'language': '0', 'question': 'Soll die automatische Gesichtserkennung im öffentlichen Raum verboten werden?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32268', 'category': '11461', 'language': '0', 'question': 'Soll die Schweiz das Schengen-Abkommen mit der EU kündigen und wieder verstärkte Personenkontrollen direkt an der Grenze einführen? [BePart-Frage]', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32269', 'category': '11462', 'language': '0', 'question': 'Befürworten Sie engere Beziehungen zur Europäischen Union (EU)?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32270', 'category': '11462', 'language': '0', 'question': 'Soll die Schweiz ein umfassendes Freihandelsabkommen (inkl. Landwirtschaft) mit den USA anstreben?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32271', 'category': '11462', 'language': '0', 'question': 'Sollen Schweizer Unternehmen, deren Tochterfirmen oder Zulieferer im Ausland tätig sind, zur Einhaltung von Sozial- und Umweltstandards verpflichtet werden?', 'rapide': '0', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32272', 'category': '11462', 'language': '0', 'question': 'Soll die Schweiz die Bilateralen Verträge mit der EU kündigen und ein Freihandelsabkommen ohne Personenfreizügigkeit anstreben?', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32273', 'category': '11462', 'language': '0', 'question': 'Soll die Schweiz zu einer strikten Auslegung der Neutralität zurückkehren (weitgehender Verzicht auf wirtschaftliche Sanktionen)? [BePart-Frage]', 'rapide': '1', 'type': '4-options', 'ID_election': '1057'},
    {'ID_question': '32274', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Wer sich nichts zuschulden kommen lässt, hat von staatlichen Sicherheitsmassnahmen nichts zu befürchten.', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32275', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Von einer freien Marktwirtschaft profitieren langfristig alle. ', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32276', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Es ist notwendig, dass der Staat durch Umverteilung die Einkommens- und Vermögensunterschiede ausgleicht.', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32277', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Für ein Kind ist es am besten, wenn ein Elternteil Vollzeit für die Kinderbetreuung zu Hause bleibt.', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32278', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Die fortschreitende Digitalisierung bietet mehr Chancen als Risiken.', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32279', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Die Bestrafung Krimineller ist wichtiger als deren Wiedereingliederung in die Gesellschaft.', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32280', 'category': '11463', 'language': '0', 'question': 'Wie beurteilen Sie folgende Aussage: Ein stärkerer Umweltschutz ist notwendig, auch wenn er zu Lasten des Wirtschaftswachstums durchgesetzt werden muss.', 'rapide': '0', 'type': '7-options', 'ID_election': '1057'},
    {'ID_question': '32281', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Soziale Wohlfahrt mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32282', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Bildung und Forschung mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32283', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Landwirtschaft mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32284', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Öffentlicher Verkehr mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32285', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Strassenverkehr (motorisierter Individualverkehr) mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32286', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Öffentliche Sicherheit mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32287', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Landesverteidigung mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'},
    {'ID_question': '32288', 'category': '11464', 'language': '0', 'question': 'Soll der Bund im Bereich Entwicklungszusammenarbeit mehr oder weniger ausgeben?', 'rapide': '0', 'type': '5-options', 'ID_election': '1057'}]

In [ ]:
for candidate in candidates:
    for answer in candidate["answers"]:
        question_id = answer["questionId"]
        question_text = next(question["question"] for question in fragen if question["ID_question"] == question_id)
        answer['text'] = question_text

Um die Daten weiterverwenden zu können, speichern wir die Resultate in einer eigenen Datei ab.


In [ ]:
# Write the response data to a file
with open(filename, "w", encoding="utf-8") as json_file:
    json.dump(candidates, json_file, indent=4, ensure_ascii=False)

print(f"Response has been written to {filename}.")